# Constraint Satisfaction

The setup to this problem is based upon the Zebra Puzzle (https://en.wikipedia.org/wiki/Zebra_Puzzle)

## Constraints and variables pertaining to the problem:

    Variables (Name): {Domain}
      - Pet (Xp): {dog, flying squirrel, chinchilla, horse, zebra}
      - Drink (Xd): {coffee, tea, milk, orange juice, water}
      - House Color (Xc): {red, green, white, yellow, blue}
      - Nationality (Xn): {british, spanish, ukrainian, norwegian, japanese}
      - Sport (Xs): {tennis, soccer, swimming, running, skiing}
      - Position (Xpos): {1, 2, 3, 4, 5}
      Variables represent the attributes of each house
     
    Constraints
       1 (Xn == british) <==> (Xc == red)
       2 (Xn == spanish) <==> (Xp == dog)
       3 (Xd == coffee ) <==> (Xc == green)
       4 (Xn == ukrainian) <==> (Xd == tea)
       5 (Xc == green) -to the right of> (Xc == white)
       6 (Xs == tennis) <==> (Xp == flying squirrel)
       7 (Xs == soccer) <==> (Xc == yellow)
       8 (Xd == milk) <==> (Xpos == 3)
       9 (Xn == norway) <==> (Xpos == 1)
      10 (Xs == swimming) <==> (Xp == chinchilla)
      11 (Xc == yellow) <==> (Xp == horse)
      12 (Xs == running) <==> (Xd == orange juice)
      13 (Xn == japanese) <==> (Xs == skiing)
      14 (Xn == norwegian) <==> (Xc == blue)
      
## Constraint Graph

![Constraint Graph](cg.png)


# 4-Queens Problem

In [1]:
import sys
sys.path.append("C:\pyGM_home")

import pyGM as gm
import numpy as np
import matplotlib.pyplot as plt


def setFactor(f, d):
    for a in range(d):
        for b in range(d):
            f[a, b] = 0.0 if (abs(int(f.vars[0]) -  int(f.vars[1])) == abs(a-b) or a == b) else 1.0
    return f;
            

def get4QueensConstraints():
    n = 4
    q = [gm.Var(i, n) for i in range(n)]
    print("Queen variables: {}".format(q))

    f = []

    _f = gm.Factor([q[0], q[1]], 1.0)
    _f = setFactor(_f, n)
    f.append(_f)
    
    _f = gm.Factor([q[0], q[2]], 1.0)
    _f = setFactor(_f, n)
    f.append(_f)
    
    _f = gm.Factor([q[0], q[3]], 1.0)
    _f = setFactor(_f, n)
    f.append(_f)
    
    _f = gm.Factor([q[1], q[2]], 1.0)
    _f = setFactor(_f, n)
    f.append(_f)
    
    _f = gm.Factor([q[1], q[3]], 1.0)
    _f = setFactor(_f, n)
    f.append(_f)
    
    _f = gm.Factor([q[2], q[3]], 1.0)
    _f = setFactor(_f, n)
    f.append(_f)
    
    for fij in f:
        for v in range(n):
            fij[v,v] = 0.0
            
    return q, f



Q, factors = get4QueensConstraints()

Queen variables: [Var (0,4), Var (1,4), Var (2,4), Var (3,4)]


We can now test our constraint model!

In [2]:
F = lambda x:np.product( [ fij[ tuple(x[v] for v in fij.vars) ] for fij in factors ] );

def checkConfiguration(config):
    if (F(config)): print(config, "is a valid configuration")
    else: print(config, "is NOT a valid configuration")
        
checkConfiguration([1,3,0,2]) # configuration from slides
checkConfiguration([2,0,3,1]) # tests
checkConfiguration([3,1,0,2])
checkConfiguration([0,2,3,1])

[1, 3, 0, 2] is a valid configuration
[2, 0, 3, 1] is a valid configuration
[3, 1, 0, 2] is NOT a valid configuration
[0, 2, 3, 1] is NOT a valid configuration


## Biased Coins

I have a bag containing n unbiased coins. N-1 of these coins are normal and one coin is fake with heads on both sides.

    C is whether or not the coin is fake
        f --> fake
        r --> real
    F is the result of the flip
        h --> heads
        t --> tails
    P(C = f) = 1/n
    P(F = h|C = f) = 1

I reach in the bag and pull out a random coin, flip it, and get heads. What is the probability that the coin I chose is biased?

    P(C = f|F = h) = P(F = h|C = f) * P(C = f) / P(F = h)
                    = 1 * (1 / n) / sum( P(F = h, C = c) )
                    = (1 / n) / (1 * (1 / n) + 0.5 * ((n-1) / n))
                    = 2 / (n + 1)

I flip the coin k times after picking it and see k heads. What is the probability that the coin I chose is biased?

    P(C = f|F = kh) = P(F = kh|C = f) * P(C = f) / P(F = kh)
                    = 1 * (1 / n) / sum( P(F = kh, C = c) )
                    = (1 / n) / (1 * (1/n) + 2^(-k)*(n-1)/n)
                    = 2^k/(2^k+n-1)

## Automative Warnings

Here is the problem setup:

    E is whether or not the engine is too hot
        0 --> okay
        1 --> too hot
    C is whether or not the coolant is too low
        0 --> okay
        1 --> too low
    W is whether or not the warning light it on
        0 --> off
        1 --> on
        
    P(E = 1) = 0.1
    P(C = 1) = 0.1
    P(W = 1|E = 1, C = 1) = 0.9
    P(W = 1|(E = 1 || C == 1)) = 0.8
    P(W = 1|E = 0, C = 0) = 0.1
    P(E = 1, C = 1) = 0.1 * 0.1 = 0.01
    P((E = 1 || C == 1)) = P(E = 1) * (1 - P(C = 1)) + (1 - P(E = 1)) * P(C = 1)
                         = 0.1 * 0.9 + 0.1 * 0.9
                         = 0.18
    P(E = 0, C = 0) = 1 - (P(E = 1) + P(C = 1) - P(E = 1) * P(C = 1))
                    = 1 - (0.1 + 0.1 - 0.1 * 0.1)
                    = 0.81
    
We can use the above information to determine P(W)
    
    P(W) = 0.9 * 0.01 + 0.8 * 0.18 + 0.1 * 0.81
         = 0.234

Now we notice that the coolant is low! Let's calculate the new probability.

    P(C = 1|W = 1) = P(W = 1| C = 1) * P(C = 1) / P(W = 1)
                   = 0.8 * 0.1 / 0.234
                   = 0.342


We have additionally figured out that the engine is too hot! Time to find a new probability.

    P(C = 1|W = 1, E = 1) = P(W = 1, E = 1|C = 1) P(C = 1) / P(W = 1, E = 1)
                          = 0.9 * 0.1 / (0.9 * 0.8)
                          = 0.111